In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

from scipy.optimize import minimize, rosen, rosen_der

## Group Assignment
### Team Number: 06
### Team Member Names: Shayan Jalali, Krish Suryavanshi, Paul Reddy
### Team Strategy Chosen: Market Meat

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.

## Ticker Filtering

The code below contains the logic used to clean and filter the raw tickers before building the portfolio. 

* We begin by importing a list of tickers from a CSV file. We first check whether each ticker has any historical data available in the specified date range. Only tickers with non-empty price data are retained.

* Next, we check if they are liquid. Illiquid stocks produce unreliable return estimates and unrealistic trading behavior, so we check for a minimum average daily trading volume of 5,000 shares. A month with fewer than 18 trading days is not considered representative and is excluded from the calculation.

* Once the valid months are identified, we calculate the stock’s average daily volume using only the trading days from these months. If the resulting average volume is at least 5,000 shares, the ticker is accepted; otherwise, it is removed.

Finally, we construct a DataFrame of daily closing prices for all tickers that passed both screens.

In [2]:
# Krish's code

csv_file = "Tickers_Example.csv"
tickers = pd.read_csv(csv_file, header=None, names=["Ticker"])

# Include only valid US and Canadian companies that have listed stocks in these markets.

start_date = "2024-10-01"
end_date = "2025-10-01" # TO INCLUDE SEPT 30TH

valid_tickers = []

for ticker in tickers["Ticker"]:

    hist = yf.Ticker(ticker).history(start=start_date, end=end_date)
    
    if not hist.empty:
        valid_tickers.append(ticker)

# Exclude any stock with average daily volume < 5,000 shares between Oct 1, 2024 and Sep 30, 2025 (drop months with < 18 trading days).

volume_cutoff = 5000
minimum_trading_days = 18
final_tickers = []

for valid_ticker in valid_tickers:

    hist = yf.Ticker(valid_ticker).history(start=start_date, end=end_date)
    months = list(hist.index.strftime("%Y-%m"))

    unique_months = []
    for month in months:
        if month not in unique_months:
            unique_months.append(month)
    
    good_months = []
    for unique_month in unique_months:
        if months.count(unique_month) >= minimum_trading_days:
            good_months.append(unique_month)
    
    mask = hist.index.strftime("%Y-%m").isin(good_months)
    filtered_hist = hist[mask]

    avg_volume = filtered_hist["Volume"].mean()
    if avg_volume >= volume_cutoff:
        final_tickers.append(valid_ticker)

# create closing prices DataFrame for the filtered tickers

close_prices = pd.DataFrame()
for final_ticker in final_tickers:
    hist = yf.Ticker(final_ticker).history(start=start_date, end=end_date)
    close_prices[final_ticker] = hist["Close"]

close_prices.index = close_prices.index.strftime("%Y-%m-%d")

close_prices

$CELG: possibly delisted; no timezone found
$MON: possibly delisted; no timezone found
$MON: possibly delisted; no timezone found
$RTN: possibly delisted; no timezone found
$RTN: possibly delisted; no timezone found


,AAPL,ABBV,ABT,ACN,AGN.CN,AIG,AMZN,AXP,BA,BAC,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-01,224.944031,188.798172,110.925453,345.618652,0.0800,71.177109,185.130005,265.044800,154.220001,38.318245,...,162.130692,160.253296,105.980003,21.057087,81.035965,193.983368,570.918274,239.846207,125.324356,42.547169
2024-10-02,225.510849,188.663986,111.003601,349.850769,0.0800,70.912834,184.759995,267.334076,152.889999,38.328018,...,164.614792,159.121506,106.830002,20.899042,80.413269,195.744217,579.737061,237.697327,125.079842,42.346203
2024-10-03,224.407043,187.350739,109.626320,354.711304,0.0800,70.805161,181.960007,265.034912,150.520004,38.347557,...,165.201584,157.817062,105.980003,20.713108,80.941628,194.050705,580.167664,235.040512,122.926392,42.087822
2024-10-04,225.530716,186.238815,110.026794,355.693207,0.0825,74.436455,186.509995,273.028839,155.000000,39.187782,...,165.182022,159.409225,111.910004,20.555063,81.620941,195.051422,578.650635,233.311615,123.424774,42.776844
2024-10-07,220.449326,185.586990,110.730095,350.351532,0.0800,72.077583,180.800003,270.872040,155.910004,39.041229,...,163.265182,159.121506,110.190002,20.601547,82.017197,193.665817,571.172791,231.367828,123.377769,42.556740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-24,252.065643,218.415421,133.297256,239.080002,0.0700,77.900002,220.210007,339.619476,215.100006,51.700001,...,173.550003,203.245789,206.839996,21.969999,107.572151,182.808289,351.809998,230.360001,83.900002,48.657986
2025-09-25,256.621216,216.975800,132.719818,232.559998,0.0700,77.099998,218.149994,339.320221,213.529999,51.849998,...,169.679993,202.332642,199.960007,21.820000,108.929314,180.429520,345.559998,232.000000,82.580002,48.509575
2025-09-26,255.212601,219.030991,132.958771,238.970001,0.0700,77.980003,219.779999,340.816437,221.259995,52.209999,...,169.199997,202.114273,195.399994,21.830000,109.315651,182.917328,344.079987,235.199997,83.720001,48.915222


## Benchmark Construction and Distance Calculation

In this section, we construct the benchmark average used to evaluate how closely each stock tracks the S&P 500 and TSX.

* We get daily closing prices for both indices over the selected timeframe. From these prices, we compute daily percentage returns and then convert them into cumulative total return series. 

* Next, we take the simple average of the cumulative return series of the S&P 500 and the TSX.

* Finally, we calculate the cumulative total returns of every filtered stock in our dataset. We then merge each stock’s cumulative return series with the benchmark. For each stock, we compute the average absolute distance between its cumulative return curve and the cumulative return of the benchmark. 
  * This metric indicates how closely a stock tracks the S&P and TSX.

The final output is a sorted table of tickers ranked by their average distance from the benchmark. This ranking can be used to identify stocks with low tracking error or to construct diversified yet benchmark-aligned portfolios.



$$
\begin{align*}
\text{Let:} \quad & R_t^{\text{SP}} = \text{cumulative return of S\&P 500} \\
& R_t^{\text{TSX}} = \text{cumulative return of TSX} \\
& R_t^j = \text{cumulative return of stock $j$} \\[1mm]
\text{Benchmark Average:} \quad & B_t = \frac{R_t^{\text{SP}} + R_t^{\text{TSX}}}{2} \\
\text{Distance Between Stock and Benchmark:} \quad & D_j = \frac{1}{T} \sum_{t=1}^{T} \big| R_t^j - B_t \big|
\end{align*}
$$



In [3]:
# Paul's code

#Get the history of the S&P500 and TSX
sp_data = yf.Ticker("^GSPC").history(start=start_date, end=end_date)
tsx_data = yf.Ticker("^GSPTSE").history(start=start_date, end=end_date)
sp_close = sp_data["Close"]
tsx_close = tsx_data["Close"]

#Calculate cumulative total returns 
sp_returns = sp_close.pct_change()
tsx_returns = tsx_close.pct_change()
sp_total_return = sp_returns.cumsum()*100
tsx_total_return = tsx_returns.cumsum()*100

#Average
benchmark_avg = (sp_total_return + tsx_total_return) / 2

#Store them in dataframes
sp_df = pd.DataFrame({"S&P500 TotalReturn": sp_total_return})
tsx_df = pd.DataFrame({"TSX TotalReturn": tsx_total_return})
benchmark_df = pd.DataFrame({"Benchmark Average": benchmark_avg})

benchmark_df.index = benchmark_df.index.strftime("%Y-%m-%d")

print("Benchmark Average:")

#Calculate cumulative total returns for stocks
stock_returns = close_prices.pct_change()
stock_total_returns = stock_returns.cumsum() * 100


#Merge the returns of the stocks and the benchmark average
merged = pd.merge(stock_total_returns, benchmark_df, on="Date", how="inner")

#Store the average distances between the returns of the stock to the returns of the benchmark average in a distances dataframe
distances_dict = {}
for ticker in close_prices.columns:
    avg_distance = (merged[ticker] - merged["Benchmark Average"]).abs().mean()
    distances_dict[ticker] = avg_distance

distances_df = pd.DataFrame(list(distances_dict.items()), columns=["Ticker", "Average Distance"])
distances_df = distances_df.sort_values(by="Average Distance", ascending=True).reset_index(drop=True)
distances_df



Benchmark Average:


/var/folders/hk/4jtzk4tx5kgbr895rh0gtm8r0000gn/T/ipykernel_93062/3423409461.py:28: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  stock_returns = close_prices.pct_change()


,Ticker,Average Distance
0,RY.TO,1.616307
1,BLK,4.284492
2,USB,4.455741
3,AXP,6.071995
4,AIG,6.400784
5,AAPL,6.909023
6,ABT,7.203754
7,ABBV,7.267377
8,QCOM,7.435317
9,T.TO,7.546121


## Portfolio Construction

The goal of this code is to construct a well-diversified portfolio of 20 stocks that balances exposure between large-cap and small-cap companies while also ensuring that no single sector dominates the portfolio. 

Note: Large-cap stocks are defined as having a market capitalization of at least 10 billion CAD, while small-cap stocks are defined as having a market capitalization below 2 billion CAD. 
<br>
<br>
* The process begins by retrieving the current USD/CAD exchange rate. Converting all values to CAD ensures consistency when applying market capitalization thresholds to categorize the stocks.

* Next, the code identifies large-cap and small-cap stocks from the pre-ranked distances_df dataframe, based on their tracking performance relative to the benchmarks. 

* As the algorithm iterates through the ranked list of stocks, it keeps track of how many stocks have already been selected from each sector to follow the sector diversity rules, ensuring that no sector constitutes more than 40% of the total portfolio. 
  * Stocks that would cause a sector to exceed this limit are skipped.

* Finally, for every stock, it retrieves the sector, market capitalization, and categorizes the stock as large, mid, or small-cap. This information is stored in a DataFrame, which forms the basis of the final portfolio composition.
<br>
<br>


Before finalizing the portfolio, the code validates that the portfolio satisfies all constraints: 
* There must be at least one large-cap and one small-cap stock, 
* No sector may exceed 40% of the portfolio. 

If any of these requirements are not met, the code outputs a warning and does not display the portfolio composition. 
If all constraints are satisfied, the portfolio composition, including the number of large-cap and small-cap stocks and their respective sectors, is displayed.

In [4]:
# *************** LOOK INTO THIS

# Shayan's code portfolio construction 

# get live USD/CAD exchange rate
exchange_rate_ticker = yf.Ticker("CAD=X")
exchange_rate_data = exchange_rate_ticker.history(period="1d") # takes the most recent trading day 
exchange_rate = exchange_rate_data['Close'].iloc[-1] # takes most recent close price from the data with [-1]
print(f"Current USD/CAD exchange rate: {exchange_rate}")

# identify large and small cap stocks from distances dataframe with sector diversity
temp_large = []
temp_small = []
sector_counts = {}  # track how many stocks from each sector
max_per_sector = 8  # 40% of 20 stocks = 8 stocks max per sector

# loop through all stocks ranked by benchmark tracking to find proper market cap mix
for ticker in distances_df['Ticker']:
    try:
        # get market cap info for each ticker
        stock = yf.Ticker(ticker)
        mc = stock.info.get('marketCap', 0)
        curr = stock.info.get('currency', 'USD')
        sector = stock.info.get('sector', 'Unknown')
         
        # convert to CAD if necessary
        mc_cad = mc * exchange_rate if curr == 'USD' else mc
        
        # check if this sector is already at limit
        if sector_counts.get(sector, 0) >= max_per_sector:
            continue  # skip this stock, sector is full
        
        # categorize by market cap size
        if mc_cad >= 10e9:  # 10 billion
            temp_large.append(ticker)
            sector_counts[sector] = sector_counts.get(sector, 0) + 1
        elif mc_cad < 2e9:  # 2 billion
            temp_small.append(ticker)
            sector_counts[sector] = sector_counts.get(sector, 0) + 1
        
        # stop once we have enough of each type
        if len(temp_large) >= 15 and len(temp_small) >= 5:
            break
    except:
        continue

# select 15 large caps and 5 small caps for total of 20 stocks
selected_tickers = temp_large[:15] + temp_small[:5]
print(f"Selected {len(selected_tickers)} stocks for portfolio")

# make empty lists to store the data that will get added into them through the loops
ticker_list = []
sector_list = []
market_cap_cad_list = []
cap_type_list = []

# for each selected ticker get detailed info and append to lists
for ticker in selected_tickers:
    try:
        # get the ticker info and extract sector, market cap, and currency
        stock = yf.Ticker(ticker)
        info = stock.info
        
        sector = info.get('sector', 'Unknown')
        market_cap = info.get('marketCap', 0)
        currency = info.get('currency', 'USD')
        
        # if the currency is usd convert to cad using exchange rate from yfinance
        if currency == 'USD':
            market_cap_cad = market_cap * exchange_rate
        else:
            market_cap_cad = market_cap
        
        # categorize market cap size
        if market_cap_cad >= 10000000000:  # 10 billion
            cap_type = 'Large'
        elif market_cap_cad < 2000000000:  # 2 billion
            cap_type = 'Small'
        else:
            cap_type = 'Mid'
        
        # add the necessary info to the lists used in the dataframe
        ticker_list.append(ticker)
        sector_list.append(sector)
        market_cap_cad_list.append(market_cap_cad)
        cap_type_list.append(cap_type)
        
    except:
        continue

# create dataframe with the ticker info, sector info, market cap in canadian info, and the cap type info
stock_info = pd.DataFrame({
    'Ticker': ticker_list,
    'Sector': sector_list,
    'MarketCap_CAD': market_cap_cad_list,
    'CapType': cap_type_list
})

# check market cap requirements
large_count = len(stock_info[stock_info['CapType'] == 'Large'])
small_count = len(stock_info[stock_info['CapType'] == 'Small'])
                                                                                                                                                                             
if large_count == 0 or small_count == 0:
    print(f"Cannot build portfolio: need at least 1 large-cap and 1 small-cap")
    print(f"Current mix: {large_count} large-cap, {small_count} small-cap")
else:
    # check sector concentration
    sector_counts = stock_info.groupby('Sector').size()
    max_sector_pct = (sector_counts.max() / len(stock_info)) * 100
    
    if max_sector_pct > 40:
        max_sector = sector_counts.idxmax() # returns index of first occurence of max for the sector_counts
        print(f"Cannot build portfolio: {max_sector} sector is {max_sector_pct:.1f}% (exceeds 40% limit)")
    else:
        # display stock composition only if all requirements met
        print(f"\nPortfolio composition: {large_count} large-cap, {small_count} small-cap stocks")
        print(stock_info)

Current USD/CAD exchange rate: 1.4019999504089355
Selected 16 stocks for portfolio
Selected 16 stocks for portfolio
Cannot build portfolio: Financial Services sector is 43.8% (exceeds 40% limit)
Cannot build portfolio: Financial Services sector is 43.8% (exceeds 40% limit)


## Relationships between the Stocks and the Benchmark

The purpose of the code below is to prepare the data for portfolio optimization by calculating the relationships between the selected stocks and the benchmark.

* Firstly, we extract the daily returns for our selected stocks and calculate benchmark daily returns as the average of the S&P and TSX. Then both datasets are aligned to the common dates

* Using the aligned stock returns, the code calculates a covariance matrix. This matrix is a 20×20 NumPy array where each element represents the covariance between a pair of stocks. This matrix captures how each stock moves relative to every other stock in the portfolio.

Note: While the code is designed to calculate a 20×20 covariance matrix for the selected portfolio of stocks, the actual size of the covariance matrix can vary depending on data availability. Some stocks may have missing return data for certain dates or may be removed during preprocessing steps such as dropping NaN values. As a result, the number of stocks included in the covariance matrix, and therefore its dimensions, may be less than 20×20.


* For each stock, the covariance with the benchmark is computed. These covariances indicate how closely each stock moves with the benchmark.

* Finally, the variance of the aligned benchmark returns is calculated. This measures the daily variability of the benchmark itself.



<div style="text-align: left;">

$$
\begin{align*}
\text{Let } & r_t^j \text{ be the daily return of stock } j \text{ on day } t, \quad j = 1, \dots, 20 \\ 
\text{Let } & r_t^{B} = \frac{r_t^{\text{SP}} + r_t^{\text{TSX}}}{2} \text{ be the benchmark daily return} \\[1mm]
\text{Aligned Returns:} \quad & \text{Consider only dates } t \in \mathcal{T}_{\text{common}} \text{ where both stock and benchmark returns exist} \\[1mm]
\text{Stock Covariance Matrix:} \quad & \Sigma = \Big[ \text{Cov}(r^i, r^j) \Big]_{i,j=1}^{20} \\[1mm]
\text{Covariance with Benchmark:} \quad & \text{cov}_{jB} = \text{Cov}(r^j, r^B), \quad j = 1, \dots, 20 \\[1mm]
\text{Benchmark Variance:} \quad & \sigma_B^2 = \text{Var}(r^B) \\[1mm]
\text{General Formula for Covariance:} \quad & \text{Cov}(X,Y) = \frac{1}{n-1} \sum_{k=1}^{n} (X_k - \bar{X})(Y_k - \bar{Y})
\end{align*}
$$

</div>


In [5]:

# get the daily returns for the selected stocks, and dropping the rows that dont
selected_stock_returns = stock_returns[selected_tickers].dropna()

# gets the benchmark daily returns: going to be useful when using scipy optimization
benchmark_daily_returns = (sp_returns + tsx_returns) / 2

# have to use strftime otherwise get mismatch when comparing to the aligned stock returns index
benchmark_daily_returns.index = benchmark_daily_returns.index.strftime('%Y-%m-%d')

# find the cmommon dates to use with .index intersection() for both the stock returns and benchmark returns (had to align them)
common_dates = selected_stock_returns.index.intersection(benchmark_daily_returns.index)
aligned_stock_returns = selected_stock_returns.loc[common_dates]
aligned_benchmark_returns = benchmark_daily_returns.loc[common_dates]


# using the covariance on alisgned selected stock rteurns to generate matrix
cov_matrix = aligned_stock_returns.cov().values # with the data for now, matrix is 20x20 numpy array with covariances between all pairs of selected stocks
# print(cov_matrix)

# calculate covariances between each stock and the benchmark
stock_benchmark_covariances = []

# for stock in the list of stocks we've filtered out
for ticker in selected_tickers:
    # assign series (1D array) to the common dates of the stock
    stock_returns_series = aligned_stock_returns[ticker]
    # assign series (1D array) to the common dates of the benchmark returns
    benchmark_returns_series = aligned_benchmark_returns

    # calucluate the covariance at that specific point (daily values)
    covariance = stock_returns_series.cov(benchmark_returns_series)
    # add it to stock_benchmark relationship
    stock_benchmark_covariances.append(covariance)

# store as numpy array for scipy on all the covariances that we've appdned to the empty list  before for loop (expects num array and also want consistensy with cov_matrix)
benchmark_cov = np.array(stock_benchmark_covariances)
    
# of the benchmark returns that are aligned with the common dates, how much does it go up and oown daily 
benchmark_var = aligned_benchmark_returns.var()

# print to verify all three covariance components
print("Covariance matrix shape:", cov_matrix.shape) # how the stocks move relative to eadch other 
print("Benchmark covariances length:", len(benchmark_cov)) # how the stocks move with the benchmark
print("Benchmark variance:", benchmark_var) # how the becnhmark moves

Covariance matrix shape: (16, 16)
Benchmark covariances length: 16
Benchmark variance: 9.698255889153167e-05


## Portfolio Optimization Using Tracking Error Variance

This code defines and solves a portfolio optimization problem that is based on the concept of tracking error variance. <br>
Tracking error quantifies how much a portfolio’s returns deviate from a benchmark.<br>
By minimizing tracking error, the goal is to construct a portfolio that closely follows the performance of the benchmark while respecting diversification constraints.

<br>
<br>


$$
\text{Portfolio Variance:} \quad \sigma_p^2 = \mathbf{w}^\top \Sigma \mathbf{w}
$$

where 𝑤 is the vector of portfolio weights. This expression captures how the weighted combination of the selected stocks fluctuates, taking into account both individual stock volatilities and correlations between the stocks.
<br>
<br>
<br>
<br>
<br>




$$
\text{Portfolio-Benchmark Covariance:} \quad \text{cov}_{pB} = \mathbf{w}^\top \text{cov}_B
$$

This term measures how the portfolio moves relative to the benchmark. It is a weighted sum of the covariances between each stock and the benchmark, reflecting how the composition of the portfolio affects its alignment with the benchmark.

<br>
<br>
<br>
<br>
<br>


$$
\text{Tracking Error Variance:} \quad \text{TEV} = \mathbf{w}^\top \Sigma \mathbf{w} - 2 \, \mathbf{w}^\top \text{cov}_B + \sigma_B^2
$$

This formula is derived from standard portfolio theory and expresses the variance of the difference between the portfolio return and the benchmark return.
<br>
<br>
<br>
<br>
<br>
In addition to minimizing tracking error, the portfolio must satisfy several constraints:
* The weights must sum to 1, ensuring that the portfolio is fully invested. 
* Each weight also has a minimum value, which prevents any single stock from being allocated an insignificantly small proportion.
* There is a maximum weight per stock (15%) to prevent over-concentration in any individual stock.


The optimizer requires an initial guess for the portfolio weights. In this case, the code uses equal weights across all stocks as a starting point. From this starting point, the minimize function adjusts the weights to find the allocation that minimizes tracking error variance while complying to all the defined constraints and bounds.

Finally, the optimal weights are extracted from the x attribute of the optimization result object. These weights represent the portfolio allocation that best replicates the benchmark while adhering to constraints



In [6]:
# Shayan's code

# define function that calculates tracking error variance
# input: weights (array of 20 numbers)
# calculate: portfolio_var = weights^T @ cov_matrix @ weights
# calculate: portfolio_bench_cov = weights^T @ benchmark_cov
# calculate: tracking_error_var = portfolio_var - 2*portfolio_bench_cov + benchmark_var
# return: tracking_error_var

def tracking_error_variance(weights):
    # inner dot product understands how the stocks amplify each other, and outer dot product determines weightages based on this interactions
    portfolio_var = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_bench_cov = np.dot(weights, benchmark_cov)
    # https://quant.stackexchange.com/questions/35720/ex-ante-tracking-error-active-strategies-and-the-size-of-the-covariance-matrix used this interesting source for calculating tracking error
    tracking_error_var = portfolio_var - 2 * portfolio_bench_cov + benchmark_var

    return tracking_error_var


# constraint 1: all weights must sum to 1 (100%)
# constraint 2: each weight must be >= minimum (100 / (2*n) percent)
# constraint 3: each weight must be <= 0.15 (15%)

# calculate n_stocks
num_stocks = len(selected_tickers)
# calculate min_weight
min_weight_stocks = (100/(2 * num_stocks)) / 100  # convert percentage to decimal

# define constratints function

# this will verify with scipy that a function returns 0 when constraint is satsiifed 
def weights_must_sum_to_one(weights):
    return np.sum(weights) - 1

# create constraints dictionary
# scipy documentation checks type 'eq' for equality, fun is the function deifning the constraint  and is callable 
constraints = [{'type': 'eq', 'fun': weights_must_sum_to_one}]

# create bounds list

bounds = []
for ticker in selected_tickers:
# boudns requires (min, max) pairs for each element based on scipy documentation. pairs = tuples also want them to be immutasble
    bounds.append((min_weight_stocks, 0.15))

# create initial_weights arrayz; op[timizer needs a starting point before caluclating based on the bounds which have been made
# x0ndarray, shape (n,)
# Initial guess. Array of real elements of size (n,), where n is the number of independent variables.
# Reqwuired to pass in x0. Basically the initial guess based on scipy documentation

# take 1 divide it by number of stocks to get equally weighting and then make an array that has the amount of num_stocks stored within it
initial_weights = np.array([1/num_stocks] * num_stocks)

# contains everything about how result went based on all the data fed into it
optimizer_data= minimize(tracking_error_variance, initial_weights, method='SLSQP', bounds = bounds, constraints=constraints)

# using SLSQP method because it handles bounds and equality constraints efficiently
# extracts the optimal weights (x attribute) from the optimizer result object

# sintially needed the x0 value, now calling the actual .x value from the optimizer 
optimal_weights = optimizer_data.x


In [7]:
# Shayan's code

# checks if optimization worked
print("Success:", optimizer_data.success)
print("Tracking error variance:", optimizer_data.fun)
print("Total weight:", np.sum(optimal_weights))

print("\nWeights by stock:")
# make counter variable 
i = 0
# make for loop that itrates over the sleected ticker list
for ticker in selected_tickers:
    # the ticker weightage is the optimal weights index * 100 to convert to percentage for each ticker and then rounds to 2 decimal places
    print(f"{ticker}: {optimal_weights[i]*100:.2f}%")
    i += 1

Success: True
Tracking error variance: 2.9732174382962937e-05
Total weight: 1.0

Weights by stock:
RY.TO: 6.25%
BLK: 6.25%
USB: 6.25%
AXP: 6.25%
AIG: 6.25%
AAPL: 6.25%
ABT: 6.25%
ABBV: 6.25%
QCOM: 6.25%
T.TO: 6.25%
TD.TO: 6.25%
TXN: 6.25%
CAT: 6.25%
AMZN: 6.25%
BAC: 6.25%
AGN.CN: 6.25%


In [8]:
# Shayan's code

print(optimal_weights)

[0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]


In [9]:
# for step 5: initial check of like constrains of 40% was to identify if the stocks selected were diverse enoughh
# after optimization, we're going to get weightages htat violate those retsirtcions so redoing the 40% restrictions here will help with ensuring that they fit the constrants
# even if those contsraints make our weightages worse, we have to follow the restrictions 

# krish's code

# make a df for tickers and weights
dict = {"Ticker": selected_tickers, "Weight": optimal_weights}
weights_df = pd.DataFrame(dict)

# create portfolio df
portfolio_df = pd.merge(weights_df, stock_info , on="Ticker", how="inner")

# get sector weights
sector_weights_series = portfolio_df.groupby("Sector")["Weight"].sum()
sector_weights = sector_weights_series.to_dict()

# print them
for sector in sector_weights:
    print(str(sector) + ": " + str(np.round(sector_weights[sector] * 100, 2)) + "%")

print()

# check for sector violations (max 40% in one sector)
sector_violations = False
for sector in sector_weights:
    if sector_weights[sector] > 0.4:
        sector_violations = True
        print("Sector weight violation: " + str(sector) + " is " + str(np.round(sector_weights[sector] * 100, 2)) + "%; max 40%")

if not sector_violations:
    print("No sector weight violations.")

print()

# check for cap violations (need one large cap and one small cap minimum)
large_rows = portfolio_df[portfolio_df["CapType"] == "Large"]
large_weight_total = large_rows["Weight"].sum()

small_rows = portfolio_df[portfolio_df["CapType"] == "Small"]
small_weight_total = small_rows["Weight"].sum()

print("Total Large Cap Weight: " + str(np.round(large_weight_total * 100, 2)) + "%")
print("Total Small Cap Weight: " + str(np.round(small_weight_total * 100, 2)) + "%")
print()

if (not large_weight_total > 0) or (not small_weight_total > 0):
    print("Market cap weight violation; need at least one large cap and one small cap stock")

Communication Services: 6.25%
Consumer Cyclical: 6.25%
Financial Services: 43.75%
Healthcare: 18.75%
Industrials: 6.25%
Technology: 18.75%

Sector weight violation: Financial Services is 43.75%; max 40%

Total Large Cap Weight: 93.75%
Total Small Cap Weight: 6.25%



In [10]:
# After the loop, before selected_tickers line:
print(f"Found {len(temp_large)} large caps")
print(f"Found {len(temp_small)} small caps")
print(f"Sector distribution: {sector_counts}")

selected_tickers = temp_large[:15] + temp_small[:5]

Found 33 large caps
Found 1 small caps
Sector distribution: Sector
Communication Services    1
Consumer Cyclical         1
Financial Services        7
Healthcare                3
Industrials               1
Technology                3
dtype: int64


In [11]:
# Paul's code

#create a dictionary with the ticker and its price and currency so we can convert for fees
price_data = {
    "Ticker": [],
    "Price": [],
    "Currency": []
}

for index, row in portfolio_df.iterrows():
    ticker = row["Ticker"]   

    try:
        stock = yf.Ticker(ticker)

        # get Nov 21, 2025 close price
        hist = stock.history(start="2025-10-21", end="2025-10-22")

        if hist.empty:
            print(f"No price data for {ticker} on Oct 21, 2025")
            continue

        close_price = hist["Close"].iloc[0]

        # extract currency
        currency = stock.info.get("currency", "Unknown")

        # add row to the df structure
        price_data["Ticker"].append(ticker)
        price_data["Price"].append(close_price)
        price_data["Currency"].append(currency)


    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# convert to DataFrame
prices_df = pd.DataFrame(price_data)

print("\nFinal Prices DataFrame:")
prices_df



Final Prices DataFrame:


,Ticker,Price,Currency
0,RY.TO,203.910797,CAD
1,BLK,1130.000000,USD
2,USB,47.610001,USD
3,AXP,355.220001,USD
4,AIG,78.379997,USD
5,AAPL,262.515503,USD
6,ABT,127.540001,USD
7,ABBV,231.389999,USD
8,QCOM,168.830002,USD
9,T.TO,21.100000,CAD


In [12]:
# calculate shares with transaction fees

# merge prices into portfolio
portfolio_with_prices = pd.merge(portfolio_df, prices_df, on="Ticker", how="inner")

# start with a budget of 1 million CAD
budget_cad = 1000000

# calculate initial target values and shares prior to fees
portfolio_with_prices ["TargetValue_CAD"] = portfolio_with_prices ["Weight"] * budget_cad

# convert prices to CAD if necessary
target_values_stock_currency = []
for index, row in portfolio_with_prices.iterrows():
    if row["Currency"] == "USD":
        target_values_stock_currency.append(row["TargetValue_CAD"] / exchange_rate)
    else:
        target_values_stock_currency.append(row["TargetValue_CAD"])

# add column with Target Values in CAD to merged portfolio
portfolio_with_prices["TargetValue_StockCurrency"] = target_values_stock_currency

# calculate shares
portfolio_with_prices["Shares"] = (portfolio_with_prices["TargetValue_StockCurrency"] / portfolio_with_prices["Price"])

# calculate fees in USD for each stock ($2.15 USD flat or 0.1% of the shares you buy whichever is smaller)
fees_usd=[]
for index, row in portfolio_with_prices.iterrows():
    fee = min(2.15, 0.001 * row["Shares"])
    fees_usd.append(fee)

portfolio_with_prices["Fee_USD"] = fees_usd

# convert all fees to CAD (fees are always calculated in USD)
fees_cad = []
for index, row in portfolio_with_prices.iterrows():
    # All fees start in USD, so always multiply by exchange rate
    fees_cad.append(row["Fee_USD"] * exchange_rate)

portfolio_with_prices["Fee_CAD"] = fees_cad

# sum total fees in CAD
total_fees_cad = portfolio_with_prices["Fee_CAD"].sum()
print(f"\nTotal Transaction Fees (CAD): ${total_fees_cad:.2f}")

# caLculate investable amount after fees
investable_amount_cad = budget_cad - total_fees_cad

# scale the investment to account for fees from initial investable amount
scaling_factor = investable_amount_cad / budget_cad
print(f"Scaling Factor after Fees: {scaling_factor:.6f}")
print(f"Investable amount after fees (CAD): ${investable_amount_cad:.2f}")

# take the final target values and adjust based on scaling factor
portfolio_with_prices["FinalTargetValue_CAD"] = portfolio_with_prices["TargetValue_CAD"] * scaling_factor

# convert the scaled down target values to the stock currency
final_target_values_stock_currency = []
for index, row in portfolio_with_prices.iterrows():
    if row["Currency"] == "USD":
        final_target_values_stock_currency.append(row["FinalTargetValue_CAD"] / exchange_rate)
    else:
        final_target_values_stock_currency.append(row["FinalTargetValue_CAD"])

portfolio_with_prices["FinalTargetValue_StockCurrency"] = final_target_values_stock_currency

# realculate shares with the scaled target values
portfolio_with_prices["FinalShares"] = (portfolio_with_prices["FinalTargetValue_StockCurrency"] / portfolio_with_prices["Price"])

# calculate the final value of each position in the stock currency
portfolio_with_prices["FinalValue_StockCurrency"] = portfolio_with_prices["FinalShares"] * portfolio_with_prices["Price"]

# convert the final values to CAD
final_values_cad = []
for index, row in portfolio_with_prices.iterrows():
    if row["Currency"] == "USD":
        final_values_cad.append(row["FinalValue_StockCurrency"] * exchange_rate)
    else:
        final_values_cad.append(row["FinalValue_StockCurrency"])

portfolio_with_prices

portfolio_with_prices["FinalValue_CAD"] = final_values_cad

# calculate the final weights
portfolio_with_prices["FinalWeight"] = portfolio_with_prices["FinalValue_CAD"] / investable_amount_cad

total_portoflio_value = portfolio_with_prices["FinalValue_CAD"].sum()
total_weights = portfolio_with_prices["FinalWeight"].sum()

print(f"\nTotal Portfolio Value (CAD): ${total_portoflio_value:.2f}")
print(f"Total of Final Weights: {total_weights:.6f}")
print(f"Difference from the investable amount: ${abs(total_portoflio_value - investable_amount_cad):.2f}")

# displayt final portfolio
print("\nFinal Portfolio:")
print(portfolio_with_prices[["Ticker", "Price", "Currency",  "FinalShares", "FinalValue_CAD", "FinalWeight"]])



Total Transaction Fees (CAD): $12.91
Scaling Factor after Fees: 0.999987
Investable amount after fees (CAD): $999987.09

Total Portfolio Value (CAD): $999987.09
Total of Final Weights: 1.000000
Difference from the investable amount: $0.00

Final Portfolio:
    Ticker        Price Currency    FinalShares  FinalValue_CAD  FinalWeight
0    RY.TO   203.910797      CAD     306.502618    62499.193218       0.0625
1      BLK  1130.000000      USD      39.450087    62499.193218       0.0625
2      USB    47.610001      USD     936.328464    62499.193218       0.0625
3      AXP   355.220001      USD     125.495745    62499.193218       0.0625
4      AIG    78.379997      USD     568.749685    62499.193218       0.0625
5     AAPL   262.515503      USD     169.813204    62499.193218       0.0625
6      ABT   127.540001      USD     349.526411    62499.193218       0.0625
7     ABBV   231.389999      USD     192.655685    62499.193218       0.0625
8     QCOM   168.830002      USD     264.044295  

In [13]:
# create portfolio with the required columns
final_portfolio = portfolio_with_prices[["Ticker", "Price", "Currency", "FinalShares", "FinalValue_CAD", "FinalWeight"]].copy()

# rename the columns to fit requirements
final_portfolio = final_portfolio.rename(columns={
    "FinalShares": "Shares",
    "FinalValue_CAD": "Value",
    "FinalWeight": "Weight"
})

# convert the weightages to a percentage
final_portfolio["Weight"] = final_portfolio["Weight"] * 100

# reset index to start at 1
final_portfolio.index = range(1, len(final_portfolio) + 1)

# display the final portfolio
print("\nFinal Portfolio DataFrame:")
print(final_portfolio)

# print relevant data about value and weights
total_value = final_portfolio["Value"].sum()
total_weight = final_portfolio["Weight"].sum()

print(f"\nTotal Portfolio Value (CAD): ${total_value:.2f}")
print(f"Total Portfolio Weight (%): {total_weight:.2f}%")
print(f"Budget adjusted for fees (CAD): ${investable_amount_cad:.2f}")

# make a dataframe with the ticker and shares purchased 
shares_purchased_df = final_portfolio[["Ticker", "Shares"]].copy()

# export to csv
shares_purchased_df.to_csv("Stocks_Group_06.csv", index=False)

print("\nShares purchased exported to Stocks_Group_06.csv")
print(f"Portfolio contains {len(final_portfolio)} stocks.")


Final Portfolio DataFrame:
    Ticker        Price Currency         Shares         Value  Weight
1    RY.TO   203.910797      CAD     306.502618  62499.193218    6.25
2      BLK  1130.000000      USD      39.450087  62499.193218    6.25
3      USB    47.610001      USD     936.328464  62499.193218    6.25
4      AXP   355.220001      USD     125.495745  62499.193218    6.25
5      AIG    78.379997      USD     568.749685  62499.193218    6.25
6     AAPL   262.515503      USD     169.813204  62499.193218    6.25
7      ABT   127.540001      USD     349.526411  62499.193218    6.25
8     ABBV   231.389999      USD     192.655685  62499.193218    6.25
9     QCOM   168.830002      USD     264.044295  62499.193218    6.25
10    T.TO    21.100000      CAD    2962.047018  62499.193218    6.25
11   TD.TO   112.639999      CAD     554.857897  62499.193218    6.25
12     TXN   179.240143      USD     248.708788  62499.193218    6.25
13     CAT   524.650024      USD      84.968258  62499.193218 